In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

In [2]:
# 創建到達率表格
arrival_rates = np.array([
    [0.1] * 30,
    [0.7] * 30,
    [0.1] * 30,
    [0.1] * 30
])

In [3]:
# 定義參數
seat_type = ['Y', 'M', 'K', 'YM', 'YK', 'MK', 'YMK'] # 座位種類
seats = 10 # 狀態 x 的座位數
lamda = 0.9 # lamda
time_space = len(arrival_rates[0]) # 時間長度
print(time_space)

30


In [20]:
# 生成價格組合
price_set = [{'Y':800, 'M':500, 'K':450}]

# 調整範圍
price_range = {
    'Y': range(800, 805, 5),
    'M': range(500, 505, 5),
    'K': range(450, 455, 5)
}

# 生成所有可能的票價组合
combinations = list(itertools.product(price_range['Y'], price_range['M'], price_range['K']))
price_combinations = [{'Y': combo[0], 'M': combo[1], 'K': combo[2]} for combo in combinations]
print(len(price_combinations))
print(price_combinations)

1
[{'Y': 800, 'M': 500, 'K': 450}]


In [21]:
# 座位選擇機率矩陣
PS = {
    'f': {'PY(S)': 0.0, 'PM(S)': 0.0, 'PK(S)': 0.0, 'P0(S)': 1.0, 'Q(S)': 0.0, 'R(S)': 0.0},
    'Y': {'PY(S)': 0.3, 'PM(S)': 0.0, 'PK(S)': 0.0, 'P0(S)': 0.7, 'Q(S)': 0.3, 'R(S)': 240.0},
    'M': {'PY(S)': 0.0, 'PM(S)': 0.6, 'PK(S)': 0.0, 'P0(S)': 0.4, 'Q(S)': 0.6, 'R(S)': 300.0},
    'K': {'PY(S)': 0.0, 'PM(S)': 0.0, 'PK(S)': 0.5, 'P0(S)': 0.5, 'Q(S)': 0.5, 'R(S)': 225.0},
    'YM': {'PY(S)': 0.1, 'PM(S)': 0.6, 'PK(S)': 0.0, 'P0(S)': 0.3, 'Q(S)': 0.7, 'R(S)': 380.0},
    'YK': {'PY(S)': 0.3, 'PM(S)': 0.0, 'PK(S)': 0.5, 'P0(S)': 0.2, 'Q(S)': 0.8, 'R(S)': 465.0},
    'MK': {'PY(S)': 0.0, 'PM(S)': 0.4, 'PK(S)': 0.5, 'P0(S)': 0.1, 'Q(S)': 0.9, 'R(S)': 425.0},
    'YMK': {'PY(S)': 0.1, 'PM(S)': 0.4, 'PK(S)': 0.5, 'P0(S)': 0.0, 'Q(S)': 1.0, 'R(S)': 505.0}
}

In [22]:
# 創建一個空字典保存三個第三維度的陣列
# three_dimensional_dict = {}
# for seat in seat_type:
#     three_dimensional_dict[seat] = np.zeros((seats+1, time_space+1))

four_dimensional_dict = {}
for price in price_combinations:
    price_key = tuple(price.items())
    four_dimensional_dict[price_key] = {}
    for seat in seat_type:
        four_dimensional_dict[price_key][seat] = np.zeros((seats + 1, time_space + 1))

print(four_dimensional_dict)

{(('Y', 800), ('M', 500), ('K', 450)): {'Y': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0

In [23]:
# 創建 value function 矩陣
value_function = np.zeros((seats+1, time_space+1))

In [24]:
'''定義價值函數的判斷式'''
def calculate_value_function(arrival_rates, seat_type, lamda, PS, price_combinations):
    
    # 創建 value function 矩陣
    value_function = np.zeros((seats+1, time_space+1))
    
    # 用於記錄每個價值函數對應的最大座位+票價組合
    max_value_seat_types_and_price = np.empty((seats+1, time_space+1), dtype=object)
        
    # 每個時間點
    for t in range(1, time_space+1):

        # 每個狀態
        for x in range(seats+1):

            # ***每一種票價組合
            for price in price_combinations:

                price_key = tuple(price.items())

                # 根據當前價格組合生成 p_j 表
                p_j = {
                    'f': {'f': 0.0, 'Y': 0.0, 'M': 0.0, 'K': 0.0},
                    'Y': {'f': 0.0, 'Y': price['Y'], 'M': 0.0, 'K': 0.0},
                    'M': {'f': 0.0, 'Y': 0.0, 'M': price['M'], 'K': 0.0},
                    'K': {'f': 0.0, 'Y': 0.0, 'M': 0.0, 'K': price['K']},
                    'YM': {'f': 0.0, 'Y': price['Y'], 'M': price['M'], 'K': 0.0},
                    'YK': {'f': 0.0, 'Y': price['Y'], 'M': 0.0, 'K': price['K']},
                    'MK': {'f': 0.0, 'Y': 0.0, 'M': price['M'], 'K': price['K']},
                    'YMK': {'f': 0.0, 'Y': price['Y'], 'M': price['M'], 'K': price['K']}
                }
            
                # 每一種座位組合
                for seat in seat_type:
                    
                    # 座位 = 0 或時間 = 0
                    if x == 0 or t == 0:  
                        value_function[x, t] = 0
                    
                    # 其他情況
                    else:
                        j_y = lamda*PS[seat]['PY(S)']*(p_j[seat]['Y']+value_function[x-1, t-1])
                        j_m = lamda*PS[seat]['PM(S)']*(p_j[seat]['M']+value_function[x-1, t-1])
                        j_k = lamda*PS[seat]['PK(S)']*(p_j[seat]['K']+value_function[x-1, t-1])
                        j_0 = (lamda*PS[seat]['P0(S)']+1-lamda)*value_function[x, t-1]
    
                        # 計算價值函數
                        current_value = j_y + j_m + j_k + j_0
                        # four_dimensional_dict[price][seat][x,t] = j_y + j_m + j_k + j_0
                        
                        # 更新價值函數 table
                        if current_value> value_function[x, t]:
                            value_function[x, t] = current_value
                            max_value_seat_types_and_price[x, t] = (price_key, seat)  # 更新最大價值對應的座位
                        
                        # 更新四维字典
                        four_dimensional_dict[price_key][seat][x, t] = current_value
    return four_dimensional_dict, value_function, max_value_seat_types_and_price

In [28]:
'''計算價值函數'''
four_dimensional_dict, value_function, max_value_seat_types_and_price = calculate_value_function(arrival_rates, seat_type, lamda, PS, price_combinations)
# print('four_dimensional_dict Y: \n', four_dimensional_dict['Y'])
# print('four_dimensional_dict M: \n', four_dimensional_dict['M'])
# print('four_dimensional_dict K: \n', four_dimensional_dict['K'])
# print('four_dimensional_dict YM: \n', four_dimensional_dict['YM'])
# print('four_dimensional_dict YK: \n', four_dimensional_dict['YK'])
# print('four_dimensional_dict MK: \n', four_dimensional_dict['MK'])
# print('four_dimensional_dict YMK: \n', four_dimensional_dict['YMK'])
print('four_dimensional_dict: \n', four_dimensional_dict)
print('value function: \n', value_function)
print('max value seat types: \n', max_value_seat_types_and_price)

four_dimensional_dict: 
 {(('Y', 800), ('M', 500), ('K', 450)): {'Y': array([[   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ],
       [   0.        ,  216.        ,  547.785     ,  615.88305   ,
         665.5946265 ,  701.88407734,  728.37537646,  747.71402482,
         761.83123812,  772.13680383,  779.65986679,  785.15170276,
         789.16074301,  792.0873424 ,  794.22375995,  795.78334476,
         796.92184168,  797.75294443,  798.35964943,  798.80254408,
         799.12585718,  799.36187574,  799.53416929,  799.65

In [ ]:
'''定義計算 Delta V_t(x) 的函式'''
def calculate_delta_value_function(value_function):
    
    # 初始化 Delta V 函數矩陣
    delta_value_function = np.zeros((len(value_function), len(value_function[0])))
    time_space = len(value_function[0])+1
    
    # 計算每個時間點下每個狀態的 Delta V 函數
    for t in range(1, time_space): 
        for x in range(1, len(value_function)):
            
            # 計算 Delta V_t(x)
            delta_value_function[x][t-1] = value_function[x][t-1] - value_function[x-1][t-1]
    
    return delta_value_function

In [ ]:
# 計算 Delta V 函數
delta_value_function = calculate_delta_value_function(value_function)
print("Delta V: \n", delta_value_function)

In [ ]:
'''根據機率分布產生顧客'''
def generate_customer(customers, willing_to_pay):
    
    # 生成隨機亂數
    random_number = np.random.rand()
    probabilities = [0.1, 0.3, 0.3, 0.3]

    # 使用累積機率決定顧客種類
    cumulative_probability = 0
    customer_index = 0
    for probability in probabilities:
        cumulative_probability += probability
        if random_number <= cumulative_probability:
            break
        customer_index += 1
    
    # 回傳顧客種類
    customer_type = customers[customer_index]
    price = willing_to_pay[customer_index]    
    print(f"random_number: {random_number}, customer_index: {customer_index}, customer_type: {customer_type}, price: {price}")
    return customer_type, price

In [ ]:
'''使用 DP 模擬'''
total_income_list = []  # 用來存總收益的 list
customers = {0: 'f', 1: 'Y', 2: 'M', 3: 'K'} # 顧客種類
willing_to_pay = [0, 800, 500, 450] # 願支付價格

# 模擬 300 次
for sim_time in range(300):
    
    total_income = 0  # 初始總收入為 0
    remaining_seats = 10  # 初始座位數量為 10
    time_space = len(arrival_rates[0])+1 # 時間為1~30
    
    # 每次模擬的過程
    for time_point in range(1, time_space):
        
        print("-------------" ,time_point, "-----------------")
        
        # 生成顧客
        customer_type, price = generate_customer(customers, willing_to_pay)

        # 若還有剩餘座位
        if remaining_seats > 0:
            
            seat_open = max_value_seat_types[remaining_seats][32-time_point-1]
            print(f"seat open: {seat_open}, customer_type: {customer_type}, Willingness to pay: {price}")
            
            delta_v = delta_value_function[remaining_seats][32-time_point-1]
            print(f"remaining seats: {remaining_seats}, time_point: {32-time_point-1}, delta_v: {delta_v}")
            
            # 座位符合顧客種類
            if p_j[seat_open][customer_type] > 0 :
               
                # 根據 Delta V 函數決定是否賣出座位
                if delta_v < price:

                    remaining_seats -= 1
                    total_income += price        
            
            else:
                print("No suitable seat to sell.")
        else:
            print("No remaining seats.")

        print(f"Remaining Seats: {remaining_seats}, Total Income: {total_income}")
        
    total_income_list.append(total_income)

In [ ]:
average_income = np.mean(total_income_list)
print(f"Average Income after {sim_time+1} simulation:", average_income)

In [ ]:
# 繪製折線圖
plt.plot(total_income_list, linestyle='-')
plt.title('Total Income for Each Simulation')
plt.xlabel('Simulation')
plt.ylabel('Total Income')
plt.grid(True)
plt.show()

In [ ]:
'''使用 FCFS 模擬'''
total_income_list_fcfs = []  # 用來存總收益的 list

# 模擬 300 次
for sim_time in range(300):
    
    total_income = 0  # 初始總收入為 0
    remaining_seats = seats  # 初始座位數量
    
    # 每次模擬的過程
    for time_point in range(1, time_space+1):
        
        print("-------------" ,time_point, "-----------------")
        
        # 生成顧客
        customer_type, price = generate_customer(customers, willing_to_pay)

        # 若還有剩餘座位
        if remaining_seats > 0:
            
            remaining_seats -= 1
            total_income += price        
            
        else:
            print("No remaining seats.")

        print(f"Total Income: {total_income}")
        
    total_income_list_fcfs.append(total_income)

In [ ]:
# 繪製折線圖
plt.plot(total_income_list_fcfs, linestyle='-')
plt.title('Total Income for Each Simulation')
plt.xlabel('Simulation')
plt.ylabel('Total Income')
plt.grid(True)
plt.show()

In [ ]:
average_income_fcfs = np.mean(total_income_list_fcfs)
print(f"Average Income after {sim_time+1} simulation:", average_income_fcfs)

In [ ]:
# 繪製圖表
plt.figure(figsize=(10, 6))  
plt.plot(range(300), total_income_list, label='DP each simulation', alpha=0.5, color='r')
plt.plot(range(300), total_income_list_fcfs, label='FCFS each simulation', alpha=0.5, color='b')
plt.axhline(y=average_income, color='r', label='Dynamic Programming')
plt.axhline(y=average_income_fcfs, color='b', label='FCFS')
plt.xlabel('Simulation')
plt.ylabel('Total Reward')
plt.title('Total Reward per simulation')
plt.legend()  
plt.grid(True)
plt.show()